# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

5902294

Prenons une clé au hasard :

In [7]:
key_= list(bigram_phrases.vocab.keys())[200]
print(key_)

(


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

123462

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['fftn', 've', 'ndredi', '\\-', 'j', 'mars_1929', 'ejfttua', 'j', ',', 's', 'louruaus', 'da', 'l', "'", 'a', 'i', 'llitl', '11', ':', 'p', '^,', 'a', 'e', '-*', 'aa', '-', 'herjea', '-', 'poti', '^', 'or4w', '.'], ['bruxelle', '.'], ['*', 'j', '.'], ['...', 'ini', '-', 'lliwi', '.'], ['...', '<<', 'x3', '.', 'nv', 'ijaiu', '0', "*'", '*>>', ',,,,', 't', '**.'], ['abunn', '"', '.'], ['<<', 'v3', '.', 'wi', '30_centimes', 'le', 'numero', 'jls', '-', 'clitjiori', '.'], ['*', '40', '(tm)_annee', 'n', 'deg', '60', '{^', 'ajt', '.', 'pluqb', '.'], ['1', '*', '11', 'br', '>>', 'bmieu', '.', 'l', '.'], ['tejoawkmknrtje', '.'], ['iovj', 'luiea', '.'], ['68', '.', '00', '<<', 'muis', '.', '83', '.', '00', ';', '3', 'mois', '.'], ['19', '.', '00', '"', 'l', 'ua', 'an', '.'], ['i', '10', '"#*;', '6', 'm', '.', '60', 'fr', '.'], [';', 's', 'ai', '.'], ['30', '(', 'r', '.', '0', ';', 'a', "'", 'uieaiu', 'conventions_postales', '.'], ['titlsh', '.'], ['edit', '>>>>.'], ['22_h', '.', 'edit', '*', 'lh

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=3, # La taille du "contexte", ici 3 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 33.1 s
Wall time: 1min 6s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [16]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [17]:
model.wv["crise"]

array([ 3.3756452 , -1.0803405 ,  2.6873198 ,  1.1695983 ,  2.9956226 ,
        3.6269834 , -1.2256876 , -2.0992527 ,  0.7340674 ,  2.9888635 ,
       -0.8857055 ,  0.9215222 , -1.1850309 ,  2.078408  , -1.6444635 ,
        0.05789915, -1.8635154 , -2.9408257 , -3.1512058 , -2.1110535 ,
       -2.6388025 ,  2.3888667 , -0.4153475 ,  0.16669005,  0.751414  ,
        1.3572164 , -3.0000167 , -0.08326261,  0.0516395 , -1.4498218 ,
        1.8678792 , -2.4287255 ], dtype=float32)

### Calculer la similarité entre deux termes

In [18]:
model.wv.similarity("crise", "depression")

0.81257457

### Chercher les mots les plus proches d'un terme donné

In [19]:
model.wv.most_similar("crise", topn=10)

[('crise_economique', 0.9599692821502686),
 ('crise_mondiale', 0.9028064608573914),
 ('surproduction', 0.9004038572311401),
 ('crise_actuelle', 0.8961181044578552),
 ('depression_economique', 0.8928499817848206),
 ('situation_actuelle', 0.8901970982551575),
 ('situation', 0.8852964043617249),
 ('crise_economique_mondiale', 0.8835370540618896),
 ('reprise_economique', 0.8816266059875488),
 ('situation_economique', 0.8630707859992981)]

### Faire des recherches complexes à travers l'espace vectoriel

In [20]:
print(model.wv.most_similar(positive=['crise', 'depression'], negative=['politique']))

[('baisse', 0.8419935703277588), ('hausse', 0.8398526310920715), ('depression_economique', 0.8279069662094116), ('difference', 0.825614869594574), ('mortalite', 0.8167779445648193), ('recolte', 0.8088059425354004), ('surproduction', 0.8078800439834595), ('periode', 0.8050872683525085), ('temperature', 0.7963736057281494), ('regression', 0.793410062789917)]
